#### Sunday, December 22, 2024

Whelp, I am now thinking the reason function calling is failing is because LMStudio does NOT yet support function calling! I am currently using LMStudio 0.3.5 (Build 1). The current [LMStudio Web Site](https://lmstudio.ai/) says 'Function Calling is now in Beta! Try out the [latest build](https://lmstudio.ai/beta-releases)' which as of today is Version 0.3.5 Build 8, last updated on 2024-11-27, so I am gonna download this and give it a go.

LMStudio has docs about Tool User [here](https://lmstudio.ai/docs/advanced/tool-use).

Hmm so I did download this version, and when I spin it up, it shows 'Build 9'. 

Nice! Running the code using 'mistral-nemo-instruct-2407', and now I AM getting back the results I want! Great!

#### Saturday, December 21, 2024

I began this notebook using 'lmstudio-community/Qwen2.5-14B-Instruct-GGUF' but this does not appear to support function calling.
OK. Since I so far have not been able to get any local model to perform function calling, I just ran stuff against OpenAI because I wanted to see what DOES work, so then I know how a local model is supposed to work.

To see which models on HuggingFace support function calling, go [here](https://huggingface.co/models?other=function-calling)

DO NOT RUN THIS AGAIN! I WANT TO KEEP THE OUTPUT FOR FUTURE REFERENCE!

OK! So this notebook was copied from 'function_calling.ipynb' and will be used to try and find some local model that supports function calling. 

In [1]:
# Deliberately set the OPENAI_API_KEY to an invalid value to ensure that the code is not using it.
import os
os.environ['OPENAI_API_KEY'] = "Nope!"

## OpenAI Function Calling with Tool Usage

In [2]:
from openai import OpenAI
import json
from os import getenv

# client = OpenAI(api_key=getenv("OPENAI_API_KEY"))

# 4090 ... 
api_key = "LMStudio"
# lmstudio = "http://192.168.2.16:1234/v1"
lmstudio = "http://127.0.0.1:1234/v1"
# model = "qwen2.5-14b-instruct@q8_0" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF :  Qwen2.5-14B-Instruct-Q8_0.gguf
# model = "qwen2.5-14b-instruct@q4_k_m" # lmstudio-community/Qwen2.5-14B-Instruct-GGUF :  Qwen2.5-14B-Instruct-Q4_K_M.gguf
# model = "lmstudio-community/mistral-nemo-instruct-2407"   # lmstudio-community/Mistral-Nemo-Instruct-2407-GGUF : Mistral-Nemo-Instruct-2407-Q4_K_M.gguf

# model = "lmstudio-community/Llama-3-Groq-8B-Tool-Use-GGUF"

model = "lmstudio-community/Mistral-Nemo-Instruct-2407-GGUF"

# chat = ChatOpenAI(base_url=lmstudio, model=model, api_key="LMStudio",  model_kwargs={"response_format": {"type": "json_object"}})
client = OpenAI(base_url=lmstudio, api_key=api_key)

In [1]:
# Run this if you do actually want to use OpenAI, and of course, do not set the key to something bogus ... 
# from openai import OpenAI
# import json
# from os import getenv
# client = OpenAI(api_key=getenv("OPENAI_API_KEY"))


In [3]:
def schedule_meeting(date, time, attendees):
    # Connect to calendar service:
    return { "event_id": "1234", "status": "Meeting scheduled successfully!",
            "date": date, "time": time, "attendees": attendees }

OPENAI_FUNCTIONS = {
    "schedule_meeting": schedule_meeting
}

In [4]:
# Our predefined function JSON schema:
functions = [
    {
        "type": "function",
        "function": {
            "type": "object",
            "name": "schedule_meeting",
            "description": "Set a meeting at a specified date and time for designated attendees",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "format": "date"},
                    "time": {"type": "string", "format": "time"},
                    "attendees": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["date", "time", "attendees"],
            },
        },
    }
]

In [5]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": "Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob.",
    }
]

In [6]:
# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    # model="gpt-3.5-turbo-1106",
    # model="gpt-3.5-turbo",
    model = model ,
    messages=messages,
    tools=functions,
    tool_choice="auto"
)

# 7.0s

In [7]:
response = response.choices[0].message
response

# response from OpenAI
# ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SPRvNtyTO6AWGqK85FMx2taW', function=Function(arguments='{"date": "2023-11-01", "time": "14:00", "attendees": ["Alice", "Bob"]}', name='schedule_meeting'), type='function')], refusal=None)
# ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pHhAr3KfObP6gPzaPNUc9oeK', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')], refusal=None)
# ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HRO0PEi7GG42M81rC5CYLRTq', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')], refusal=None)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='150521075', function=Function(arguments='{"date":"2023-11-01","time":"14:00:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')])

In [8]:
print(response.tool_calls)

# response from OpenAI
# [ChatCompletionMessageToolCall(id='call_SPRvNtyTO6AWGqK85FMx2taW', function=Function(arguments='{"date": "2023-11-01", "time": "14:00", "attendees": ["Alice", "Bob"]}', name='schedule_meeting'), type='function')]
# [ChatCompletionMessageToolCall(id='call_pHhAr3KfObP6gPzaPNUc9oeK', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')]
# [ChatCompletionMessageToolCall(id='call_HRO0PEi7GG42M81rC5CYLRTq', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')]

[ChatCompletionMessageToolCall(id='150521075', function=Function(arguments='{"date":"2023-11-01","time":"14:00:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function')]


In [10]:
# Check if the model wants to call our function:
if response.tool_calls:
    # Get the first function call:
    first_tool_call = response.tool_calls[0]

    # Find the function name and function args to call:
    function_name = first_tool_call.function.name
    function_args = json.loads(first_tool_call.function.arguments)
    print("This is the function name: ", function_name)
    print("These are the function arguments: ", function_args)

    function = OPENAI_FUNCTIONS.get(function_name)

    if not function:
        raise Exception(f"Function {function_name} not found.")

    # Call the function:
    function_response = function(**function_args)

    # Share the function's response with the model:
    messages.append(
        {
            "role": "function",
            "name": "schedule_meeting",
            "content": json.dumps(function_response),
        }
    )

    # Let the model generate a user-friendly response:
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo-0613", messages=messages
    # )
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo", messages=messages
    # )

    second_response = client.chat.completions.create(
        model=model, messages=messages
    )

    print(second_response.choices[0].message.content)


# Output from 'gpt-3.5-turbo'
# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
# Meeting scheduled for November 1, 2023 at 14:00 with Alice and Bob. Event ID: 1234.


# This is the expected output ...

# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
# Meeting Scheduled:

# Date: 2023-11-01
# Time: 14:00
# Attendees: Alice, Bob

# Please mark your calendars accordingly.

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00:00', 'attendees': ['Alice', 'Bob']}


BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'role' field that is in [user, assistant, system, tool]. Got 'function'."}

--------------------------------

## Parallel Function Calling:

In [11]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": '''Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob. 
        Then I want to schedule another meeting on 2023-11-02 at 15:00 with Charlie and Dave.'''
    }
]


In [12]:
# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    # model="gpt-3.5-turbo-1106",
    # model="gpt-3.5-turbo",
    model=model,
    messages=messages,
    tools=functions,
)

# 12.1 s

In [13]:
response = response.choices[0].message
response

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='382644799', function=Function(arguments='{"date":"2023-11-01","time":"14:00","attendees":["Alice","Bob"]}', name='schedule_meeting'), type='function'), ChatCompletionMessageToolCall(id='937421461', function=Function(arguments='{"date":"2023-11-02","time":"15:00","attendees":["Charlie","Dave"]}', name='schedule_meeting'), type='function')])

In [14]:
# Check if the model wants to call our function:
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Get the function name and arguments to call:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        print("This is the function name: ", function_name)
        print("These are the function arguments: ", function_args)

        function = OPENAI_FUNCTIONS.get(function_name)

        if not function:
            raise Exception(f"Function {function_name} not found.")

        # Call the function:
        function_response = function(**function_args)

        # Share the function's response with the model:
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )

    # Let the model generate a user-friendly response:
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo-0613", messages=messages
    # )
    # second_response = client.chat.completions.create(
    #     model="gpt-3.5-turbo", messages=messages
    # )
    second_response = client.chat.completions.create(
        model=model, messages=messages
    )

    print(second_response.choices[0].message.content)

# output from 'gpt-3.5-turbo' ...
# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
# This is the function name:  schedule_meeting
# These are the function arguments:  {'date': '2023-11-02', 'time': '15:00', 'attendees': ['Charlie', 'Dave']}
# Two meetings have been successfully scheduled:
# 1. Meeting with Alice and Bob on 2023-11-01 at 14:00
# 2. Meeting with Charlie and Dave on 2023-11-02 at 15:00.

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-02', 'time': '15:00', 'attendees': ['Charlie', 'Dave']}


BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'role' field that is in [user, assistant, system, tool]. Got 'function'."}

---

## Function Calling in LangChain

In [15]:
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional

In [16]:
class Article(BaseModel):
    """Identifying key points and contrarian views in an article."""

    points: str = Field(..., description="Key points from the article")
    contrarian_points: Optional[str] = Field(
        None, description="Any contrarian points acknowledged in the article"
    )
    author: Optional[str] = Field(None, description="Author of the article")

In [17]:
_EXTRACTION_TEMPLATE = """Extract and save the relevant entities mentioned \
in the following passage together with their properties.

If a property is not present and is not required in the function parameters, do not include it in the output."""

# Create a prompt telling the LLM to extract information
prompt = ChatPromptTemplate.from_messages(
    {("system", _EXTRACTION_TEMPLATE), ("user", "{input}")}
)

In [15]:
# model = ChatOpenAI()

In [18]:
pydantic_schemas = [Article]

# Convert Pydantic objects to the appropriate schema:
tools = [convert_to_openai_tool(p) for p in pydantic_schemas]

# Give the model access to these tools:
# model = model.bind_tools(tools=tools)
client = client.bind_tools(tools=tools)

# Create an end to end chain
# chain = prompt | model | PydanticToolsParser(tools=pydantic_schemas)
chain = prompt | client | PydanticToolsParser(tools=pydantic_schemas)

AttributeError: 'OpenAI' object has no attribute 'bind_tools'

In [17]:
result = chain.invoke(
    {
        "input": """In the recent article titled 'AI adoption in industry', key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI. However, the author, Dr. Jane Smith, acknowledges a contrarian view — that without stringent regulations, AI may pose high risks."""
    }
)
print(result)

# output from openai ...
# [Article(points='Key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI.', contrarian_points='Without stringent regulations, AI may pose high risks.', author='Dr. Jane Smith')]

[Article(points='Key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI.', contrarian_points='Without stringent regulations, AI may pose high risks.', author='Dr. Jane Smith')]


---

## Extracting Chains with LangChain

In [18]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

In [19]:
# Make sure to use a recent model that supports tools
# model = ChatOpenAI(model="gpt-3.5-turbo-1106")
model = ChatOpenAI(model="gpt-3.5-turbo")

In [20]:
class Person(BaseModel):
    """A person's name and age."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")

In [21]:
chain = create_extraction_chain_pydantic(Person, model)
chain.invoke({'input':'''Bob is 25 years old. He lives in New York. He likes to play basketball. Sarah is 30 years old. She lives in San Francisco. She likes to play tennis.'''})

# output from openai ...
# [Person(name='Bob', age=25), Person(name='Sarah', age=30)]

[Person(name='Bob', age=25), Person(name='Sarah', age=30)]